In [ ]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
df = pd.read_parquet("..\..\Archivos\\business\part-00000-be1f27a4-9044-46ac-a33d-4189b2080bdb-c000.snappy.parquet")

In [ ]:
def apply_scaler(df, categorical_cols, scaler=None):

        if scaler is None:
            scaler = MinMaxScaler()

        cols = [col for col in df.columns if col not in categorical_cols]
        df[cols] = scaler.fit_transform(df[cols])

        return df

In [ ]:
X = df.drop("Label_Index", axis=1)

X = apply_scaler(X, categorical_cols=[])

X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.relu = nn.ELU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        return x

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc4 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc1 = nn.Linear(128, 4)
        self.relu = nn.ELU()

    def forward(self, x):
        x = self.relu(self.fc4(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc2(x))
        x = self.fc1(x)
        return x

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
model = Autoencoder()

# Definir el criterio y el optimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenar el autoencoder
num_epochs = 50
batch_size = 256

for epoch in range(num_epochs):
    for i in range(0, X_train.size(0), batch_size):
        batch = X_train[i:i+batch_size]
        output = model(batch)
        loss = criterion(output, batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Obtener la representación latente usando solo el encoder
with torch.no_grad():
    X_train_encoded = model.encoder(X_train).numpy()
    X_test_encoded = model.encoder(X_test).numpy()

# Aplicar K-means en la representación latente
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(X_train_encoded)

# Predecir clusters en el conjunto de prueba
clusters_train = kmeans.predict(X_train_encoded)
clusters_test = kmeans.predict(X_test_encoded)